In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
sns.set_style('whitegrid')

In [ ]:
#Load Datasets
try:
    fraud_data = pd.read_csv('../data/raw/Fraud_Data.csv')
    ip_map = pd.read_csv('../data/raw/IpAddress_to_Country.csv')
    credit_card = pd.read_csv('../data/raw/creditcard.csv')
    print("Datasets loaded successfully!")
except FileNotFoundError as e:
    print(f"Error loading datasets: {e}")
    print("Please ensure the data files are in the 'data/raw/' directory.")

In [ ]:
# Initial Inspection - Fraud_Data.csv
print("--- Fraud_Data Info ---")
fraud_data.info()
print("\n--- First 5 Rows ---")
print(fraud_data.head())
print(f"\n--- Duplicates: {fraud_data.duplicated().sum()} ---")

In [ ]:
# Data Cleaning - Fraud_Data.csv
# Correcting data types for time columns
fraud_data['signup_time'] = pd.to_datetime(fraud_data['signup_time'])
fraud_data['purchase_time'] = pd.to_datetime(fraud_data['purchase_time'])
print("\n'signup_time' and 'purchase_time' converted to datetime objects.")

# Check for missing values
print(f"\n--- Missing Values ---\n{fraud_data.isnull().sum()}")

In [ ]:
#Initial Inspection - creditcard.csv
print("\n\n--- creditcard.csv Info ---")
credit_card.info()
print("\n--- First 5 Rows ---")
print(credit_card.head())


In [ ]:
#Data Cleaning - creditcard.csv
# Check for missing values
print(f"\n--- Missing Values ---\n{credit_card.isnull().sum()}")

# Check for duplicates and remove them
num_duplicates = credit_card.duplicated().sum()
print(f"\n--- Duplicates found: {num_duplicates} ---")
if num_duplicates > 0:
    credit_card.drop_duplicates(inplace=True)
    print(f"Removed {num_duplicates} duplicates. New shape: {credit_card.shape}")


In [ ]:
# Initial Inspection - IpAddress_to_Country.csv
print("\n\n--- IpAddress_to_Country.csv Info ---")
ip_map.info()
print("\n--- First 5 Rows ---")
print(ip_map.head())

In [ ]:
#EDA - Credit Card Data - Target Variable Distribution
print("--- Credit Card Data: Class Distribution ---")
class_counts = credit_card['Class'].value_counts()
class_perc = credit_card['Class'].value_counts(normalize=True) * 100

print(f"Non-Fraudulent (0): {class_counts[0]} ({class_perc[0]:.4f}%)")
print(f"Fraudulent (1):     {class_counts[1]} ({class_perc[1]:.4f}%)")

plt.figure(figsize=(8, 6))
sns.countplot(x='Class', data=credit_card)
plt.title('Credit Card Transaction Class Distribution', fontsize=16)
plt.ylabel('Number of Transactions')
plt.xlabel('Class (0: Non-Fraud, 1: Fraud)')
plt.savefig('../reports/figures/creditcard_class_distribution.png')
plt.show()

In [ ]:
#EDA - Credit Card Data - Transaction Amount & Time Analysis
fig, axes = plt.subplots(1, 2, figsize=(18, 5))

# Plot for Transaction Amount
sns.histplot(credit_card['Amount'], bins=50, kde=True, ax=axes[0])
axes[0].set_title('Distribution of Transaction Amount')
axes[0].set_xlim(0, 1000) # Limiting for better visibility

# Plot for Time
sns.histplot(credit_card['Time'], bins=50, kde=True, ax=axes[1])
axes[1].set_title('Distribution of Transaction Time')

plt.suptitle('Amount and Time Distributions', fontsize=16)
plt.show()

# Compare Amount for Fraud vs. Non-Fraud
plt.figure(figsize=(10, 7))
sns.boxplot(x='Class', y='Amount', data=credit_card)
plt.title('Transaction Amount by Class')
plt.ylim(0, 500) # Limiting for better visibility
plt.show()

In [ ]:
#EDA - E-commerce Data - Target Variable Distribution
print("\n--- E-commerce Data: Class Distribution ---")
class_counts_ecom = fraud_data['class'].value_counts()
class_perc_ecom = fraud_data['class'].value_counts(normalize=True) * 100

print(f"Non-Fraudulent (0): {class_counts_ecom[0]} ({class_perc_ecom[0]:.2f}%)")
print(f"Fraudulent (1):     {class_counts_ecom[1]} ({class_perc_ecom[1]:.2f}%)")

plt.figure(figsize=(8, 6))
sns.countplot(x='class', data=fraud_data)
plt.title('E-commerce Transaction Class Distribution', fontsize=16)
plt.savefig('../reports/figures/ecommerce_class_distribution.png')
plt.show()

In [ ]:
#EDA - E-commerce Data - Categorical Feature Analysis
categorical_features = ['source', 'browser', 'sex']
fig, axes = plt.subplots(1, 3, figsize=(22, 6))

for i, feature in enumerate(categorical_features):
    sns.countplot(x=feature, data=fraud_data, ax=axes[i], order=fraud_data[feature].value_counts().index)
    axes[i].set_title(f'Distribution of {feature.capitalize()}')
    axes[i].tick_params(axis='x', rotation=45)

plt.suptitle('Categorical Feature Distributions', fontsize=16)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig('../reports/figures/ecommerce_categorical_distributions.png')
plt.show()

# Let's analyze fraud rate by category
for feature in categorical_features:
    fraud_rate = fraud_data.groupby(feature)['class'].mean().sort_values(ascending=False)
    print(f"\n--- Fraud Rate by {feature.capitalize()} ---")
    print(fraud_rate)

In [ ]:
#Geolocation Merge - IP Address Conversion
print("--- Starting Geolocation Analysis ---")

def ip_to_integer(ip_address):
    """Converts a dot-decimal IP string to its integer representation."""
    try:
        parts = ip_address.split('.')
        # Formula: a*(256^3) + b*(256^2) + c*(256^1) + d*(256^0)
        return int(parts[0]) * 16777216 + int(parts[1]) * 65536 + int(parts[2]) * 256 + int(parts[3])
    except:
        return np.nan # Return NaN for any malformed IPs

# Apply the conversion
fraud_data['ip_as_int'] = fraud_data['ip_address'].apply(ip_to_integer)

print(f"IP addresses converted to integer format.")
print(f"Number of IPs that could not be converted: {fraud_data['ip_as_int'].isnull().sum()}")

print(fraud_data[['ip_address', 'ip_as_int']].head())

In [ ]:
#Merging Datasets using pd.merge_asof
fraud_data_sorted = fraud_data.sort_values('ip_as_int').copy()
ip_map_sorted = ip_map.sort_values('lower_bound_ip_address').copy()

# Perform the 'asof' merge. 
merged_data = pd.merge_asof(
    left=fraud_data_sorted,
    right=ip_map_sorted,
    left_on='ip_as_int',
    right_on='lower_bound_ip_address',
    direction='backward'
)

final_merged = merged_data[merged_data['ip_as_int'] <= merged_data['upper_bound_ip_address']]

print("\n--- Merge Complete ---")
print(f"Original fraud_data rows: {len(fraud_data)}")
print(f"Rows after merging and filtering: {len(final_merged)}")
print(f"Number of IPs not matched to a country: {len(fraud_data) - len(final_merged)}")

# Let's check for any nulls in the new country column
print(f"Nulls in 'country' column: {final_merged['country'].isnull().sum()}")


In [ ]:
#Post-Merge Analysis and Cleanup
# Ttop countries for transactions
print("\n--- Top 10 Transaction Countries ---")
print(final_merged['country'].value_counts().head(10))

# Top countries for FRAUD
fraudulent_transactions = final_merged[final_merged['class'] == 1]
print("\n--- Top 10 Fraudulent Transaction Countries ---")
print(fraudulent_transactions['country'].value_counts().head(10))

# Clean up intermediate columns
final_merged = final_merged.drop(columns=['ip_as_int', 'lower_bound_ip_address', 'upper_bound_ip_address'])

# Assign the final merged dataframe back to our main variable
fraud_data = final_merged.copy()
print("\n--- Final Merged Dataframe Head ---")
print(fraud_data.head())

In [ ]:
#Feature Engineering - Time-Based Features
print("--- Starting Feature Engineering ---")

# 1. time_since_signup: Calculate duration between signup and purchase
fraud_data['time_since_signup'] = (fraud_data['purchase_time'] - fraud_data['signup_time']).dt.total_seconds()

# 2. hour_of_day: Extract the hour from the purchase time
fraud_data['hour_of_day'] = fraud_data['purchase_time'].dt.hour

# 3. day_of_week: Extract the day of the week (Monday=0, Sunday=6)
fraud_data['day_of_week'] = fraud_data['purchase_time'].dt.dayofweek

print("Created time-based features: 'time_since_signup', 'hour_of_day', 'day_of_week'")
print(fraud_data[['signup_time', 'purchase_time', 'time_since_signup', 'hour_of_day', 'day_of_week']].head())

In [ ]:
#Feature Engineering - Transaction Frequency
device_counts = fraud_data['device_id'].value_counts()
fraud_data['device_id_counts'] = fraud_data['device_id'].map(device_counts)

print("\nCreated frequency-based feature: 'device_id_counts'")
print(fraud_data[['user_id', 'device_id', 'device_id_counts']].head())

In [ ]:
#Analysis of New Features
fig, axes = plt.subplots(1, 2, figsize=(18, 6))

# Plot fraud rate by hour of day
sns.barplot(data=fraud_data, x='hour_of_day', y='class', ax=axes[0], errorbar=None)
axes[0].set_title('Fraud Rate by Hour of Day')
axes[0].set_ylabel('Fraud Rate')

# Plot fraud rate by day of week
sns.barplot(data=fraud_data, x='day_of_week', y='class', ax=axes[1], errorbar=None)
axes[1].set_title('Fraud Rate by Day of Week')
axes[1].set_xticklabels(['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])
axes[1].set_ylabel('Fraud Rate')

plt.suptitle('Fraud Patterns by Time', fontsize=16)
plt.savefig('../reports/figures/ecommerce_fraud_rate_by_time.png')
plt.show()

# Analyze time_since_signup for fraud vs. non-fraud
plt.figure(figsize=(10, 7))
sns.boxplot(x='class', y='time_since_signup', data=fraud_data)
plt.title('Time Since Signup for Fraud vs. Non-Fraud Transactions')
plt.ylabel('Time Since Signup (seconds)')
plt.savefig('../reports/figures/ecommerce_time_since_signup.png')
plt.show()

In [ ]:
#Final Cleanup before Transformation
fraud_data_processed = fraud_data.drop(columns=['signup_time', 'purchase_time'])

fraud_data_processed = fraud_data_processed.drop(columns=['ip_address', 'device_id'])

fraud_data_processed = fraud_data_processed.drop(columns=['user_id'])

print("\n--- Final E-commerce Dataframe for Modeling ---")
print(fraud_data_processed.head())
print(f"\nRemaining columns: {fraud_data_processed.columns.tolist()}")

credit_card_processed = credit_card.copy()

In [ ]:
from sklearn.preprocessing import StandardScaler

#Data Transformation - E-commerce Data
print("--- Starting Data Transformation for E-commerce Data ---")

X_ecom = fraud_data_processed.drop('class', axis=1)
y_ecom = fraud_data_processed['class']

categorical_cols = X_ecom.select_dtypes(include=['object']).columns
numerical_cols = X_ecom.select_dtypes(include=np.number).columns

print(f"Categorical columns: {list(categorical_cols)}")
print(f"Numerical columns: {list(numerical_cols)}")

X_ecom_encoded = pd.get_dummies(X_ecom, columns=categorical_cols, drop_first=True)

scaler_ecom = StandardScaler()
X_ecom_scaled_numerical = scaler_ecom.fit_transform(X_ecom[numerical_cols])

X_ecom_encoded[numerical_cols] = X_ecom_scaled_numerical

ecommerce_final = pd.concat([X_ecom_encoded, y_ecom], axis=1)

print("\nE-commerce data transformed successfully.")
print("Final shape:", ecommerce_final.shape)
print(ecommerce_final.head())


In [ ]:
#Data Transformation - Credit Card Data
print("\n--- Starting Data Transformation for Credit Card Data ---")

scaler_cc = StandardScaler()
credit_card_processed[['Time', 'Amount']] = scaler_cc.fit_transform(credit_card_processed[['Time', 'Amount']])

creditcard_final = credit_card_processed.copy()

print("\nCredit card data transformed successfully.")
print("Final shape:", creditcard_final.shape)
print(creditcard_final.head())


In [ ]:
#Save Processed Data
ecommerce_final.to_csv('../data/processed/ecommerce_processed.csv', index=False)
creditcard_final.to_csv('../data/processed/creditcard_processed.csv', index=False)

print("\n--- Processed data saved successfully! ---")
print("Saved 'ecommerce_processed.csv' and 'creditcard_processed.csv' to 'data/processed/'.")